In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [3]:
import zipfile
with zipfile.ZipFile("Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("")

In [4]:
archive="./Data for test"
import pathlib
data_dir = pathlib.Path(archive).with_suffix('')
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

300


In [5]:
batch_size = 32
img_height = 224
img_width = 224

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
      labels='inferred',
    label_mode='categorical',
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 300 files belonging to 3 classes.
Using 240 files for training.


In [7]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
        labels='inferred',
    label_mode='categorical',
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 300 files belonging to 3 classes.
Using 60 files for validation.


In [8]:
class_names = train_ds.class_names
print(class_names)

['Bed', 'Chair', 'Sofa']


In [9]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 224, 224, 3)
(32, 3)


In [10]:
labels_batch

<tf.Tensor: shape=(32, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)>

In [11]:
#preprocess_input = tf.keras.applications.InceptionV3.preprocess_input
#preprocess_input = tf.keras.applications.mobilenet_v2.
preprocess_input = tf.keras.applications.resnet.preprocess_input

In [12]:
# base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
IMG_SIZE = (224,224)
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.resnet.ResNet152(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


234698864/234698864 [==============================] - 2s 0us/step


In [13]:
image_batch, label_batch = next(iter(train_ds))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 7, 7, 2048)


In [14]:
base_model.trainable = False

In [15]:
# Let's take a look at the base model architecture
base_model.summary()

Model: "resnet152"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                          

In [16]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(32, 2048)


In [17]:
prediction_layer = tf.keras.layers.Dense(3,activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(32, 3)


In [18]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])


In [19]:
inputs = tf.keras.Input(shape=(224,224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [20]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [21]:
len(model.trainable_variables)

2

In [22]:
initial_epochs = 25

loss0, accuracy0 = model.evaluate(val_ds)

2/2 [==============================] - 7s 870ms/step - loss: 2.0393 - accuracy: 0.3667


In [23]:
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)

Epoch 1/25


8/8 [==============================] - 18s 514ms/step - loss: 2.0823 - accuracy: 0.3333 - val_loss: 1.6674 - val_accuracy: 0.3833
Epoch 2/25
8/8 [==============================] - 7s 446ms/step - loss: 1.6843 - accuracy: 0.3125 - val_loss: 1.3859 - val_accuracy: 0.3833
Epoch 3/25
8/8 [==============================] - 6s 453ms/step - loss: 1.5907 - accuracy: 0.3458 - val_loss: 1.2102 - val_accuracy: 0.4333
Epoch 4/25
8/8 [==============================] - 6s 450ms/step - loss: 1.3363 - accuracy: 0.3958 - val_loss: 1.1160 - val_accuracy: 0.5000
Epoch 5/25
8/8 [==============================] - 7s 455ms/step - loss: 1.3008 - accuracy: 0.4125 - val_loss: 1.0423 - val_accuracy: 0.4667
Epoch 6/25
8/8 [==============================] - 6s 484ms/step - loss: 1.1682 - accuracy: 0.4292 - val_loss: 0.9768 - val_accuracy: 0.5167
Epoch 7/25
8/8 [==============================] - 7s 460ms/step - loss: 1.1015 - accuracy: 0.5500 - val_loss: 0.9190 - val_accuracy: 0.5833
Epoch 8/25
8/8 [==============

In [28]:
tf.keras.models.save_model(model,'my_model1.h5')

In [25]:
# Saving the model for Future Inferences

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [35]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
#from tensorflow.keras.applications. import preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet import preprocess_input
import numpy as np
#load saved model
model_bro = load_model('my_model.h5') 
img_path = 'fresh.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds=model_bro.predict(x)
# create a list containing the class labels
class_labels = ['Apple','Banana','Orange']
# find the index of the class with maximum score
pred = np.argmax(preds, axis=-1)
# print the label of the class with maximum score
print(class_labels[pred[0]])

1/1 [==============================] - 3s 3s/step
Banana


In [ ]:
#second

In [ ]:
base_model = tf.keras.applications.DenseNet201(include_top=False, weights='imagenet')


74836368/74836368 [==============================] - 5s 0us/step


In [ ]:
learning_rate_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.0003, 
                                                                         decay_steps=2, 
                                                                         decay_rate=0.97, 
                                                                         staircase=False)

In [ ]:
try:
    os.mkdir("../tmp")
except:
    print("Directory /tmp already exists")

checkpoint = tf.keras.callbacks.ModelCheckpoint("../tmp/multi_class_weights_1", 
                                                monitor='val_loss', 
                                                verbose=1, 
                                                save_best_only=False,
                                                save_weights_only=True, 
                                                period=1)

Directory /tmp already exists


In [ ]:
def dense_net_model(trainable_weights=False, weights_path=None):
    
    tf.keras.backend.clear_session()
    
    dense_net = tf.keras.applications.DenseNet201(input_shape=(256, 256, 3), weights="imagenet", include_top=False)
    
    for layer in dense_net.layers:
        layer.trainable=trainable_weights
    
    model = tf.keras.models.Sequential([dense_net,
                                        tf.keras.layers.GlobalAveragePooling2D(),
                                        tf.keras.layers.Dense(128, activation='relu'),
                                        tf.keras.layers.Dropout(0.3),
                                        tf.keras.layers.Dense(3, activation='softmax')
                                ])
    
    if weights_path:
        model.load_weights(weights_path)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_scheduler)
    
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    
    return model

dense_net_transfer = dense_net_model()
dense_net_transfer_history = dense_net_transfer.fit(train_ds, validation_data=val_ds, epochs=25, steps_per_epoch=32)

Epoch 1/25


ValueError: ignored